# Donkey Colab 3.1.5 with GPU(TensorFlow 1.13.1)

Donkey Colab with GPU.
- Version 20200826(2020年8月26日)
- DonkeyCar version 3.1.5(2020年8月25日時点)
- TensorFlow 1.13.1

## 概要
Google Colabを使用し、Donkey Carの学習を行います

上からコードをを実行することで、以下のことができます
- Google Colabの環境設定
- 学習
- 学習結果の取得  




# TensorFlowのインストール

TensorFlowのVersionは、DonkeyCarのRaspberryPi3にはいっているバージョンと同じバージョンをインストールします。  
Google Colabはデフォルトでtensorflow-2.3.0、CUDA 10.1が入っています（2020年8月24日）。  \
なので一度アンインストールし、バージョンを揃えています。

In [ ]:
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.13.1

# TensorFlowのバージョン確認

In [ ]:
!pip freeze | grep -i -e tensorflow

# GPUの確認

"Found GPU at: /device:GPU:0"と表示されれば、GPUが使用できる状態です。

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# DonkeyCarのインストール
Raspberry PiにインストールしてあるDonkeyCarとバージョンを合わせておきます。

In [ ]:
!git clone https://github.com/autorope/donkeycar.git 
%cd donkeycar
!git checkout be45ae038cb00d81c5450b9a481dbf5aaa03e688
!pip3 install -e .[pc]

# DonkeyCarのProjectを作成

In [ ]:
!donkey createcar --path /content/mycar

## myconfig.pyを編集します。
* CACHE_IMAGESをFalseにします。これは学習中のメモリ消費量を抑えるための設定です。
* GYM_CONF["guid"]をコメントアウトします。これは20200825現在、バーチャルレースを主体に開発が進んでいるために有効化しているようですが、学習時には不要です。

In [ ]:
!sed -i 's/# CACHE_IMAGES = True/CACHE_IMAGES = False/g' /content/mycar/myconfig.py
!sed -i 's/use_multiprocessing = False/use_multiprocessing = True/g' /content/mycar/train.py
!sed -i 's/^GYM_CONF/# GYM_CONF/g' /content/mycar/myconfig.py

# data.zipのアップロードとデータの解凍
Google Driveを経由しない場合のdata.zipのアップロード方法になります。  
この方法はGoogle Chromeで動作します。  
data.zip（DonkeyCarで集めた学習用データ）をアップロードします。  
下記コードを実行すると、ファイルのアップロード フォームが出現します。

ファイルは`/content/mycar/`以下に送信され、data.zipの解凍を行っています。\
データセットのディレクトリ構造は、`/content/mycar/data/tub_1_xxxx`のようになります。

Google Drive経由でアップロードを早く完了させることも出来ます。\
参考
* その他：Google Drive経由でdata.zipをアップロードする場合

In [ ]:
%cd /content/mycar/

import os
import shutil
from google.colab import files

if(os.path.exists("/content/mycar/data.zip")):
   os.remove("/content/mycar/data.zip")
if(os.path.exists("/content/mycar/data")):
   shutil.rmtree("/content/mycar/data")

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_path = "/content/mycar/" + file_name
print("-----------------------------------------------------")
print("【処理結果】%sにデータをアップロードしました。" % file_path)
print("-----------------------------------------------------")
!unzip -o data.zip

#学習の開始

Colabの環境構築が出来たので、DonkeyCarの学習を行います。
<br>
<br>
学習の対象は、/content/mycar/data/に存在するフォルダ全部になります。


In [ ]:
type = "linear"
trained_mode = "/content/mycar/models/mypilot.h5"

%cd /content/mycar
!python manage.py train --model={trained_mode} --type={type}

# 学習結果の確認
DonkeyCarは学習済みモデルを解析して動画に出力する機能があります。

学習したモデルがどのようになったのかを確認します。

モデル毎にレイヤーが全く異なるため、makemovieを使えるモデルはlinearモデルのみです。  

h5pyのバージョンも2.xが必要になるので、ここでインストールしておきます。  
**`【RESTART RUNTIME】`ボタンは無視してください！**

### モデル可視化用パッケージをインストール

In [ ]:
!pip uninstall -y keras-vis
!pip install git+https://github.com/sctse999/keras-vis
!apt install -y ffmpeg
!pip install h5py==2.10.0

### 動画ファイルの作成

複数のtubには対応していないため、`--tub=tub_xxxx`オプションを使って1つのデータディレクトリを指定する必要があります。

In [ ]:
!ls /content/mycar/data

表示されたデータディレクトリのうち、1つを決めて`tub_xxxx`の部分に書き換えます。\
データ件数がそれほど多くないディレクトリだとすぐに確認できます。

In [ ]:
mp4_path = 'tub_movie.mp4'
!donkey makemovie --tub=data/tub_1_xxxx --model models/mypilot.h5 --type linear --salient --out=$mp4_path

### 動画のダウンロード
作成した解析動画をPCにダウンロードします。\
ダウンロードした動画をPCで再生すると、モデルがどこに反応しているのか確認することができます。\
緑色の線：人の操作。青色の線：AIの判断。

In [ ]:
from google.colab import files

files.download('/content/mycar/'+mp4_path)

# 学習結果の転送
学習結果は、/content/mycar/models/mypilot.h5に保存されます。  下記セルを実行し、H5ファイルをダウンロードします。

RaspberryPiの~/mycar/models/mypilot.h5に学習結果をコピーすることで、DonkeyCarの自動走行ができるようになります。

In [ ]:
from google.colab import files

files.download('/content/mycar/models/mypilot.h5')

# その他：Google Drive経由でdata.zipをアップロードする場合
ブラウザ経由でデータをアップロードすると、とても遅いため、Google Drive経由でアップロードを早く完了させたい場合に使います。  
下記コードを実行すると、認証コードを使用してランタイム上に Google ドライブをマウントします。  
`Go to this URL in a browser: http://...`と出ているので、まずはこのリンクをクリックして、アクセスの許可をおこないます。  
許可後に`このコードをコピーし、アプリケーションに切り替えて貼り付けてください。`という表示と共に文字列が表示されているので、それを`Enter your authorization code:`に入力してリターンキーを押します。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/My Drive' | grep data 

Google Driveは、/content/drive/MyDriveにマウントされます。

data.zipファイルを/content/mycar/以下にコピーして、解凍をおこないます。

Google Driveの任意のフォルダにアップロードした場合は、'/content/drive/MyDrive/任意のフォルダ名/data.zip' のように置き換えてください。

In [ ]:
%cd /content/mycar/

import os
import shutil

if(os.path.exists("/content/mycar/data.zip")):
   os.remove("/content/mycar/data.zip")
if(os.path.exists("/content/mycar/data")):
   shutil.rmtree("/content/mycar/data")

!ls -l '/content/drive/MyDrive/data.zip'
!cp '/content/drive/MyDrive/data.zip' /content/mycar/data.zip
!unzip data.zip

# その他：特定のデータディレクトリのみを指定して学習させる場合

特定のデータディレクトリのみを指定して学習させることも出来ます。\
lsコマンドで、/content/mycar/dataに存在するデータセットのディレクトリを確認します。

tub_xxxxの部分を、指定したいディレクトリ名に変更してください。カンマ「,」で複数のディレクトリを指定することが出来ます。

In [ ]:
!ls /content/mycar/data

In [ ]:
model_type = "linear" # linear|categorical|rnn|imu|behavior|3d|localizer|latent
trained_mode = "/content/mycar/models/mypilot.h5"
tub_path = "/content/mycar/data/tub_1_xxxx,/content/mycar/data/tub_2_xxxx"

%cd /content/mycar
!python manage.py train --model={trained_mode} --type={model_type} --tub={tub_path}

# その他：CUDAバージョンに問題がある場合
Colabは突然パッケージバージョンが変更されていることがあります。\
CUDAバージョンが変更になると、Tensorflow 1.13.1が利用出来なくなるため、そのような状況になった場合は以下を実行して既存のCUDAをアンインストール後、適切なCUDAをインストールします。

In [ ]:
!apt uninstall -y cuda
!apt autoremove -y
!apt update
!apt install -y cuda-10-0